# Data retrieval from localbitcoins API

In [1]:
import requests
import pandas as pd
import numpy as np
import datetime as dt
from datetime import datetime, timezone

In [2]:
def make_request(url_base,url):
    r = requests.get(url_base+url)
    file_json = r.json()
    return file_json

def json_to_pandas(file_json:dict):
    return pd.DataFrame.from_dict(file_json)


### Calculation exchange rate of reference to further math on pct_change:
def exchange_reference(df, date_reference):
    
    memory = {}
        
    for currency_code in df["currency_code"].unique():
        
        if currency_code != "VED":
            date_reference = date_reference
        else:
            date_reference = "2021-10-10"

    
        ## Select currency for calculations:
        df_exchange = df[df['currency_code'] == currency_code ]

        ## Select date to calculate reference

        df_date_reference = df_exchange.loc[(df_exchange.date).dt.date == pd.Timestamp(date_reference)]

        ### Averare exchange rate for day of reference:

        implicit_exchange_reference = df_date_reference.implicit_exchange.mean()
        
        memory[currency_code] = implicit_exchange_reference

    ### Percentual Variation

#     equation = ((df_exchange.implicit_exchange / implicit_exchange_reference)-1)*100
    
    
    return memory

## Currencies

In [3]:
url_base = 'https://localbitcoins.com/'
# Currencies:
url = "api/currencies"
currency_code = make_request(url_base,url)
df_currency_code = json_to_pandas(currency_code['data'])
# df_currency_code.info()
# df_currency_code.head()


In [4]:
## Extract country information related to currency
currency_name = []
currency_is_altcoin = []
for index, row in df_currency_code.iterrows():
    currency_name.append(row[0]["name"])
    currency_is_altcoin.append(row[0]["altcoin"])
    
## Reorganize on df

df_currency_code['currencies'] = currency_name
df_currency_code['altcoin'] = currency_is_altcoin
df_currency_code = df_currency_code.reset_index()

df_currency_code.head()

,index,currencies,currency_count,altcoin
0,ADA,Cardano (ADA),183,True
1,AED,United Arab Emirates Dirham (AED),183,False
2,AFN,Afghan Afghani (AFN),183,False
3,ALL,Albanian Lek (ALL),183,False
4,AMD,Armenian Dram (AMD),183,False


### Average price of btc by country (in local currency)

In [5]:
url = "bitcoinaverage/ticker-all-currencies/"
price_btc = make_request(url_base,url)
time_stamp = datetime.now().timestamp()

In [6]:
df_price_btc = json_to_pandas(price_btc)

#transpose data frame. Index is now by Country:

df_price_btc = df_price_btc.T
df_price_btc['time_stamp'] = time_stamp
df_price_btc = df_price_btc.reset_index()
print(df_price_btc.info())
df_price_btc.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76 entries, 0 to 75
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   index       76 non-null     object 
 1   volume_btc  76 non-null     object 
 2   rates       76 non-null     object 
 3   avg_1h      42 non-null     object 
 4   avg_6h      67 non-null     object 
 5   avg_12h     73 non-null     object 
 6   avg_24h     76 non-null     object 
 7   time_stamp  76 non-null     float64
dtypes: float64(1), object(7)
memory usage: 4.9+ KB
None


,index,volume_btc,rates,avg_1h,avg_6h,avg_12h,avg_24h,time_stamp
0,RUB,4.65002363,{'last': '4309162.28'},4374045.62,4372118.42,4357036.97,4340906.66,1.635780e+09
1,VED,3.19548228,{'last': '268441.96'},2051518926.86,5869154853.95,5824580561.78,7683216214.10,1.635780e+09
2,USD,1.91735950,{'last': '67299.02'},61873.00,61570.38,61524.69,61271.82,1.635780e+09
3,EUR,1.80634114,{'last': '60592.64'},54707.90,54227.37,54278.43,54217.82,1.635780e+09
4,MYR,0.86579404,{'last': '261262.97'},264776.95,250982.18,251840.87,251272.54,1.635780e+09


## Merge and clean dataframe

In [7]:
df_market = df_price_btc.merge(df_currency_code, how='right')

# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

# Add date from time_stamp
df_market['date'] = datetime.fromtimestamp(time_stamp, tz=timezone.utc)
df_market = df_market[['time_stamp','date','index','currencies','volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','altcoin']]
df_market.rename(columns = {'index':'currency_code'}, inplace = True)
print(df_market.info())
df_market.head(10)


### I should include some data formatting so info is stored in the right format

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 1 to 181
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     76 non-null     float64            
 1   date           76 non-null     datetime64[ns, UTC]
 2   currency_code  76 non-null     object             
 3   currencies     76 non-null     object             
 4   volume_btc     76 non-null     object             
 5   avg_24h        76 non-null     object             
 6   avg_12h        73 non-null     object             
 7   avg_6h         67 non-null     object             
 8   avg_1h         42 non-null     object             
 9   altcoin        76 non-null     bool               
dtypes: bool(1), datetime64[ns, UTC](1), float64(1), object(7)
memory usage: 6.0+ KB
None


,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin
1,1.635780e+09,2021-11-01 15:20:03.024633+00:00,AED,United Arab Emirates Dirham (AED),0.11291129,221412.76,221412.76,221412.76,NaN,False
7,1.635780e+09,2021-11-01 15:20:03.024633+00:00,ARS,Argentine Peso (ARS),0.26361259,12149133.70,12264415.18,12264415.18,12399035.58,False
8,1.635780e+09,2021-11-01 15:20:03.024633+00:00,AUD,Australian Dollar (AUD),0.88448487,82722.35,83887.33,89035.54,NaN,False
14,1.635780e+09,2021-11-01 15:20:03.024633+00:00,BDT,Bangladeshi Taka (BDT),0.06100102,5123282.86,5132611.78,5198568.88,NaN,False
20,1.635780e+09,2021-11-01 15:20:03.024633+00:00,BOB,Bolivian Boliviano (BOB),0.03138313,438483.99,438492.21,438492.21,437806.48,False
21,1.635780e+09,2021-11-01 15:20:03.024633+00:00,BRL,Brazilian Real (BRL),0.60661741,349218.66,348637.25,348630.89,344151.12,False
25,1.635780e+09,2021-11-01 15:20:03.024633+00:00,BYN,Belarusian Ruble (BYN),0.10503966,152810.76,153025.95,154375.52,152194.21,False
27,1.635780e+09,2021-11-01 15:20:03.024633+00:00,CAD,Canadian Dollar (CAD),0.59733782,74824.90,74739.41,74890.51,75192.14,False
29,1.635780e+09,2021-11-01 15:20:03.024633+00:00,CHF,Swiss Franc (CHF),0.00642233,60045.19,60045.19,60113.29,NaN,False
31,1.635780e+09,2021-11-01 15:20:03.024633+00:00,CLP,Chilean Peso (CLP),0.45488222,51320291.00,52747652.70,52484480.00,52807309.79,False


In [8]:
## Adding avg_24h exchange rate USD/BTC for inmediate comparison:

df_market["avg_24h_usd"] = df_market[df_market['currency_code']=="USD"].avg_24h.tolist()[0]

In [9]:
### Formatting data types:

# Float values:
for column_label in ['volume_btc','avg_24h','avg_12h','avg_6h','avg_1h','avg_24h_usd']:

    df_market[column_label] = pd.to_numeric(df_market[column_label])

df_market.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76 entries, 1 to 181
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype              
---  ------         --------------  -----              
 0   time_stamp     76 non-null     float64            
 1   date           76 non-null     datetime64[ns, UTC]
 2   currency_code  76 non-null     object             
 3   currencies     76 non-null     object             
 4   volume_btc     76 non-null     float64            
 5   avg_24h        76 non-null     float64            
 6   avg_12h        73 non-null     float64            
 7   avg_6h         67 non-null     float64            
 8   avg_1h         42 non-null     float64            
 9   altcoin        76 non-null     bool               
 10  avg_24h_usd    76 non-null     float64            
dtypes: bool(1), datetime64[ns, UTC](1), float64(7), object(2)
memory usage: 6.6+ KB


In [10]:
## Calculating implicit exchange rate:
df_market['implicit_exchange'] = df_market['avg_24h'].divide(df_market["avg_24h_usd"], fill_value = None)

In [11]:
# Drop the rows with NAN values in time_stamp
df_market.dropna(subset=['time_stamp'], inplace=True)

In [12]:
df_market.head(2)

,time_stamp,date,currency_code,currencies,volume_btc,avg_24h,avg_12h,avg_6h,avg_1h,altcoin,avg_24h_usd,implicit_exchange
1,1.635780e+09,2021-11-01 15:20:03.024633+00:00,AED,United Arab Emirates Dirham (AED),0.112911,221412.76,221412.76,221412.76,NaN,False,61271.82,3.613615
7,1.635780e+09,2021-11-01 15:20:03.024633+00:00,ARS,Argentine Peso (ARS),0.263613,12149133.70,12264415.18,12264415.18,12399035.58,False,61271.82,198.282566


## Save information in .csv and sql database

In [13]:
# Saving csv
df_market.to_csv(f'./data/csv/{time_stamp}.csv',index=False)

In [14]:
# Append information on sqlite database

from sqlalchemy import Column, Integer, Float, String, DateTime,Boolean, ForeignKey
from sqlalchemy.orm import declarative_base
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

engine = create_engine('sqlite:///data/CurrenciesDataBase.db')
print(type(engine))
df_market.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>


#### Testing alternative way of getting pct_change

In [15]:
path_to_currencies = "sqlite:///data/CurrenciesDataBase.db"
df_market = pd.read_sql('currencies_vs_btc', path_to_currencies)

In [16]:
df = df_market.groupby(by=["date","currency_code"]).mean()
df.reset_index(inplace=True)

### Dataset with implicit exchange of reference by currency:
date_reference = "2021-10-03"
exchange = exchange_reference(df, date_reference)
exchange = pd.DataFrame.from_dict(exchange, orient="index")

df['pct'] = df.apply(lambda x : ((x['implicit_exchange']/exchange.loc[x["currency_code"]])-1)*100, axis = 1 )

### Drop Null values:

df.dropna(subset=['pct'], inplace=True)

/home/becode/.local/lib/python3.8/site-packages/pandas/core/ops/array_ops.py:73: FutureWarning: Comparison of Timestamp with datetime.date is deprecated in order to match the standard library behavior.  In a future version these will be considered non-comparable.Use 'ts == pd.Timestamp(date)' or 'ts.date() == date' instead.
  result = libops.scalar_compare(x.ravel(), y, op)


In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5245 entries, 0 to 5789
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   date               5245 non-null   datetime64[ns]
 1   currency_code      5245 non-null   object        
 2   time_stamp         5245 non-null   float64       
 3   volume_btc         5245 non-null   float64       
 4   avg_24h            5245 non-null   float64       
 5   avg_12h            4763 non-null   float64       
 6   avg_6h             4223 non-null   float64       
 7   avg_1h             2537 non-null   float64       
 8   altcoin            5245 non-null   float64       
 9   avg_24h_usd        5245 non-null   float64       
 10  implicit_exchange  5245 non-null   float64       
 11  pct                5245 non-null   float64       
dtypes: datetime64[ns](1), float64(10), object(1)
memory usage: 532.7+ KB


In [18]:
### Saving on DB

engine = create_engine('sqlite:///data/CurrenciesDataBase_V2.db')
print(type(engine))
df.to_sql(name="currencies_vs_btc", con=engine, if_exists="append", index=False)

<class 'sqlalchemy.engine.base.Engine'>
